------------

# Merge df_with_dummies with displacement data

In [2]:
import numpy as np
import libpysal
import libpysal.weights as lpw
import spreg
import pandas as pd
import pysal as ps
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
#df_dummies= pd.read_csv(r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\df_with_dummies.csv")
#states_gdf = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\Datasets\som_adm_ocha_itos_20230308_shp\som_admbnda_adm1_ocha_20230308.shp"

df_dummies= pd.read_csv(r"/home/sara/Documenti/GitHub/Climate-and-conflict/csv/df_lags_2016_n4_c.csv")
states_gdf = r"/home/sara/Documenti/GitHub/Climate-and-conflict/Datasets/som_adm_ocha_itos_20230308_shp/som_admbnda_adm1_ocha_20230308.shp"

In [4]:
#df = pd.read_excel(r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\displacements\UNHCR-PRMN-Displacement-Dataset - Somalia.xlsx")
df = pd.read_excel(r"/home/sara/Documenti/GitHub/Climate-and-conflict/displacements/UNHCR-PRMN-Displacement-Dataset - Somalia.xlsx")
df = df[(df['Reason']=='Drought related')]
df['Current (Arrival) Region'] = df['Current (Arrival) Region'].str.replace(' ', '_')
df['Previous (Departure) Region'] = df['Previous (Departure) Region'].str.replace(' ', '_')

In [5]:
v= df["Month End"]
v=v.values
dt = [datetime.strptime(v[i], "%d/%m/%Y") for i in range(len(v))]
q=[]

for i in range(len(dt)):
    q.append(datetime.timestamp(dt[i]))
    
df.insert(loc=3, column='date_timestamp', value=q)
df = df.sort_values("date_timestamp")

df['Month End'] = pd.to_datetime(df['Month End'], dayfirst=True)

In [6]:
aggregated_data = df.groupby([pd.Grouper(key='Month End', freq='M'),'Previous (Departure) Region', 'Current (Arrival) Region'])['Number of Individuals'].sum().to_frame()

In [7]:
dates = aggregated_data.index.get_level_values('Month End').unique()
districts = aggregated_data.index.get_level_values('Previous (Departure) Region').unique()
all_combinations = pd.MultiIndex.from_product([dates, districts,districts], names=['time', 'Previous (Departure) Region','Current (Arrival) Region'])

disp_data = aggregated_data.reindex(all_combinations, fill_value=0).reset_index()   

In [8]:
disp_matxs = disp_data.pivot_table(index=['time','Current (Arrival) Region'], columns='Previous (Departure) Region', values='Number of Individuals', aggfunc='sum').reset_index()
disp_matxs = disp_matxs.rename(columns={'Current (Arrival) Region': 'admin1'})

In [9]:
#remove the day from the date
disp_matxs['yr_mth'] = disp_matxs['time'].map(lambda x: x.strftime('%Y-%m'))
disp_matxs = disp_matxs.drop(columns=['time'])
df_dummies['yr_mth'] = pd.to_datetime(df_dummies['time'], dayfirst=True).map(lambda x: x.strftime('%Y-%m'))

df_merged = pd.merge(df_dummies, disp_matxs, on=['yr_mth', 'admin1'], how='inner')

In [10]:
#add a column with the sum of the displacements
df_merged['sum_disp'] = df_merged.iloc[:, -18:].sum(axis=1)

In [10]:
#df_merged.to_csv(r"/home/sara/Documenti/GitHub/Climate-and-conflict/csv/df_lags_2016_n4c_disp_d.csv", index=False)